In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

pd.options.display.max_colwidth = 200

import warnings
warnings.filterwarnings('ignore')


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-specialization-data/Labels_Medical_Notes.csv
/kaggle/input/nlp-specialization-data/Cleaned_POS_Medical_Notes.csv
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes/1893.txt
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes/1711.txt
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes/1773.txt
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes/1812.txt
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes/1093.txt


In [2]:
# !pip install spacy==2.3.5

In [3]:
!pip install https://med7.s3.eu-west-2.amazonaws.com/en_core_med7_lg.tar.gz

     |████████████████████████████████| 892.8 MB 5.7 kB/s  eta 0:00:01    |███▍                            | 95.1 MB 67.9 MB/s eta 0:00:12     |███████████████████████████████ | 864.5 MB 92.1 MB/s eta 0:00:01
  Created wheel for en-core-med7-lg: filename=en_core_med7_lg-0.0.3-py3-none-any.whl size=893633388 sha256=ab976f5e085521b46857e2ffa6f7a73d921e22c5dd95e5b3ee4be4dec721339c
  Stored in directory: /tmp/pip-ephem-wheel-cache-sa847t5o/wheels/9d/77/2c/365d93e14dec91a10355606b5f17c76164caa1883295fb6135
Successfully built en-core-med7-lg


### Syntax

Syntax is the structure of a language which is governed by grammers. Any ordering of words can not be a sentence. Hence, we need syntactical analysis for natural languages.

## Table of Contents

* [Parts of Speech Tagging](#pos)
* [Dependency Parsing](#parsing)
* [Named Entity Recognition](#ner)

<a id='pos'></a>

# 1. Parts of Speech Tagging

Parts of speech (POS) are specific lexical categories to which words are assigned, based on their syntactic context and role. Usually, words can fall into one of the following major categories.

* <strong>Nouns</strong>
* <strong>Verb</strong>
* <strong>Adjective</strong>
* <strong>Adverb</strong>

Besides these four major categories of parts of speech , there are other categories that occur frequently in the English language. These include pronouns, prepositions, interjections, conjunctions, determiners, and many others. The process of classifying and labeling POS tags for words called parts of speech tagging or POS tagging . POS tags are used to annotate words and depict their POS, which is really helpful to perform specific analysis, such as narrowing down upon nouns and seeing which ones are the most prominent, word sense disambiguation, and grammar analysis.


In [4]:
data = pd.read_csv('/kaggle/input/nlp-specialization-data/Cleaned_POS_Medical_Notes.csv') #for excel file use read_excel
data

,text,label,clean_text
0,\n \n \n \n \n <B>CC:</B> Difficulty with word finding.\n \n <B>HX: </B>This 27y/o RHF experienced sudden onset word finding difficulty and slurred speech on the evening of 2/19/96. She denied any...,Neurology,"difficulty word find . rhf experience sudden onset word find difficulty slurred speech evening 2/19/96 . deny associate dysphagia , diplopia , numbness weakness extremity . go sleep symptom 2/19/9..."
1,\n \n \n \n \n <B>PREOPERATIVE DIAGNOSIS: </B> Ganglion of the left wrist.\n \n <B>POSTOPERATIVE DIAGNOSIS: </B> Ganglion of the left wrist.\n \n <B>OPERATION: </B> Excision of ganglion.\n \n <B>A...,Orthopedic,"preoperative ganglion left wrist . postoperative ganglion left wrist . excision ganglion . general . estimate blood less 5 ml . successful anesthetic , patient position operating table . tournique..."
2,"\n \n \n \n \n <B>PREOPERATIVE DIAGNOSIS: </B> Cervical myelopathy, C3-4, secondary to stenosis from herniated nucleus pulposus, C3-4.\n \n <B>POSTOPERATIVE DIAGNOSES: </B> Cervical myelopathy, C3...",Orthopedic,"preoperative cervical myelopathy , c3 - 4 , secondary stenosis herniated nucleus pulposus , c3 - 4 . postoperative cervical myelopathy , c3 - 4 , secondary stenosis herniated nucleus pulposus , c3..."
3,\n \n \n \n \n <B>EXAM:</B>MRI LEFT SHOULDER\n \n <B>CLINICAL:</B>This is a 69-year-old male with pain in the shoulder. Evaluate for rotator cuff tear.\n \n <B>FINDINGS:</B>\n \n Examination was p...,Radiology,"mri leave shoulder 69-year - old male pain shoulder . evaluate rotator cuff tear . examination perform 9/1/05 . mark supraspinatus tendinosis extensive tearing substance tendon articular surface ,..."
4,\n \n \n \n \n <B>HISTORY OF PRESENT ILLNESS: </B> The patient is a 57-year-old female being seen today for evaluation of pain and symptoms related to a recurrent bunion deformity in bilateral fee...,Orthopedic,"history present patient 57-year - old female see today evaluation pain symptom relate recurrent bunion deformity bilateral foot , great leave . patient state increase symptom pain discomfort assoc..."
...,...,...,...
813,"\n \n \n \n \n <B>EXAM: </B> CT chest with contrast.\n \n <B>HISTORY: </B> Abnormal chest x-ray, which demonstrated a region of consolidation versus mass in the right upper lobe.\n \n <B>TECHNIQUE...",Radiology,"ct chest contrast . abnormal chest x - ray , demonstrate region consolidation mass right upper lobe . post contrast - enhance spiral image obtain chest . several , discrete , patchy air - space op..."
814,"\n \n \n \n \n <B>CC:</B> Episodic mental status change and RUE numbness, and chorea (found on exam).\n \n <B>HX:</B> This 78y/o RHM was referred for an episode of unusual behavior and RUE numbnes...",Radiology,"episodic mental status change numbness , chorea find exam . 78y rhm refer episode unusual behavior numbness . 9/91 , experience loss consciousness , generalize weakness , headache vomiting . evalu..."
815,"\n \n \n \n \n <B>PROCEDURE IN DETAIL: </B> Following premedication with Vistaril 50 mg and Atropine 0.4 mg IM, the patient received Versed 5.0 mg intravenously after Cetacaine spray to the poster...",Gastroenterology,"procedure follow premedication vistaril 50 mg atropine 0.4 mg , patient receive versed 5.0 mg cetacaine spray posterior palate . olympus video gastroscope introduce upper esophagus pass direct vis..."
816,\n \n \n \n \n <B>REASON FOR VISIT: </B> Overactive bladder with microscopic hematuria.\n \n <B>HISTORY OF PRESENT ILLNESS: </B> The patient is a 56-year-old noted to have microscopic hematuria wi...,Urology,reason overactive bladder microscopic hematuria . history present patient 56-year - old note microscopic hematuria overactive bladder . cystoscopy perform unremarkable . continue episode frequency...


In [5]:
import nltk
import spacy
import en_core_med7_lg #en_core_web_sm
import re

nlp=en_core_med7_lg.load()
#nlp = spacy.load('en_core_med7_lg', parse=True, tag=True, entity=True)

In [6]:
sample_text = data.clean_text.iloc[1]
print (sample_text)

preoperative ganglion left wrist . postoperative ganglion left wrist . excision ganglion . general . estimate blood less 5 ml . successful anesthetic , patient position operating table . tourniquet apply upper arm . extremity preppe usual manner surgical procedure drape . superficial vessel exsanguinate elastic wrap tourniquet inflate usual arm pressure . curved incision make present ganglion dorsal aspect wrist . blunt sharp dissection , dissect extensor tendon stalk appear arise distal radiocapitellar joint dorsal capsule excise ganglion specimen remove submit . small superficial vessel electrocoagulate instill close skin 4 0 prolene , area 6 7 ml 0.25 marcaine epinephrine . jackson - pratt drain insert tourniquet release , keep deflate 5 10 minute pass activate remove recovery room . dressing apply hand xeroform , 4x4s , abd , kerlix , elastic wrap volar fiberglass splint . tourniquet release . circulation return finger . patient allow awaken leave operating room good condition .


### WE tokenzie in the LEXICAL framework as well, but here we are looking for the process of filtering 

In [7]:
text_tokenized = nlp(sample_text)

for token in text_tokenized:
    print ("{} ---> {}".format(token,token.pos_))

preoperative ---> ADJ
ganglion ---> NOUN
left ---> VERB
wrist ---> NOUN
. ---> PUNCT
postoperative ---> ADJ
ganglion ---> NOUN
left ---> VERB
wrist ---> NOUN
. ---> PUNCT
excision ---> NOUN
ganglion ---> NOUN
. ---> PUNCT
general ---> ADJ
. ---> PUNCT
estimate ---> VERB
blood ---> NOUN
less ---> ADJ
5 ---> NUM
ml ---> NOUN
. ---> NOUN
successful ---> ADJ
anesthetic ---> NOUN
, ---> PUNCT
patient ---> ADJ
position ---> NOUN
operating ---> VERB
table ---> NOUN
. ---> PUNCT
tourniquet ---> NOUN
apply ---> VERB
upper ---> ADJ
arm ---> NOUN
. ---> PUNCT
extremity ---> NOUN
preppe ---> ADJ
usual ---> ADJ
manner ---> ADJ
surgical ---> ADJ
procedure ---> NOUN
drape ---> NOUN
. ---> PUNCT
superficial ---> ADJ
vessel ---> NOUN
exsanguinate ---> NOUN
elastic ---> ADJ
wrap ---> NOUN
tourniquet ---> NOUN
inflate ---> VERB
usual ---> ADJ
arm ---> NOUN
pressure ---> NOUN
. ---> PUNCT
curved ---> ADJ
incision ---> NOUN
make ---> VERB
present ---> ADJ
ganglion ---> NOUN
dorsal ---> ADJ
aspect ---> NOUN

Usually POS tags are used for analysis, feature engineering or, feature selection. In this analysis, let us select only the words that are nouns, verbs, number and adjectives.

In [8]:
def get_selected_pos(text):
    text_tokenized = nlp(text)
    #selecting only selected parts of speech tags
    selected_words = [token.string for token in text_tokenized if token.pos_ in ['NOUN','PROPN','NUM','ADJ','VERB','PUNCT']]
    processed_text = re.sub(' +',' ', " ".join(selected_words))
    return processed_text

In [9]:
data = data.dropna(subset=['clean_text'])
data.clean_text = data.clean_text.apply(get_selected_pos)

In [10]:
data

,text,label,clean_text
0,\n \n \n \n \n <B>CC:</B> Difficulty with word finding.\n \n <B>HX: </B>This 27y/o RHF experienced sudden onset word finding difficulty and slurred speech on the evening of 2/19/96. She denied any...,Neurology,"difficulty word find . rhf experience sudden onset word find difficulty slurred speech evening 2/19/96 . deny associate dysphagia , diplopia , numbness weakness extremity . go sleep symptom 2/19/9..."
1,\n \n \n \n \n <B>PREOPERATIVE DIAGNOSIS: </B> Ganglion of the left wrist.\n \n <B>POSTOPERATIVE DIAGNOSIS: </B> Ganglion of the left wrist.\n \n <B>OPERATION: </B> Excision of ganglion.\n \n <B>A...,Orthopedic,"preoperative ganglion left wrist . postoperative ganglion left wrist . excision ganglion . general . estimate blood less 5 ml . successful anesthetic , patient position operating table . tournique..."
2,"\n \n \n \n \n <B>PREOPERATIVE DIAGNOSIS: </B> Cervical myelopathy, C3-4, secondary to stenosis from herniated nucleus pulposus, C3-4.\n \n <B>POSTOPERATIVE DIAGNOSES: </B> Cervical myelopathy, C3...",Orthopedic,"preoperative cervical myelopathy , c3 - 4 , secondary stenosis herniated nucleus pulposus , c3 - 4 . postoperative cervical myelopathy , c3 - 4 , secondary stenosis herniated nucleus pulposus , c3..."
3,\n \n \n \n \n <B>EXAM:</B>MRI LEFT SHOULDER\n \n <B>CLINICAL:</B>This is a 69-year-old male with pain in the shoulder. Evaluate for rotator cuff tear.\n \n <B>FINDINGS:</B>\n \n Examination was p...,Radiology,"mri leave shoulder 69-year - old male pain shoulder . evaluate rotator cuff tear . examination perform 9/1/05 . mark supraspinatus tendinosis extensive tearing substance tendon articular surface ,..."
4,\n \n \n \n \n <B>HISTORY OF PRESENT ILLNESS: </B> The patient is a 57-year-old female being seen today for evaluation of pain and symptoms related to a recurrent bunion deformity in bilateral fee...,Orthopedic,"history present patient 57-year - old female see today evaluation pain symptom relate recurrent bunion deformity bilateral foot , great leave . patient state increase symptom pain discomfort assoc..."
...,...,...,...
813,"\n \n \n \n \n <B>EXAM: </B> CT chest with contrast.\n \n <B>HISTORY: </B> Abnormal chest x-ray, which demonstrated a region of consolidation versus mass in the right upper lobe.\n \n <B>TECHNIQUE...",Radiology,"ct chest contrast . abnormal chest x - ray , demonstrate region consolidation mass right upper lobe . post contrast - enhance spiral image obtain chest . several , discrete , patchy air - space op..."
814,"\n \n \n \n \n <B>CC:</B> Episodic mental status change and RUE numbness, and chorea (found on exam).\n \n <B>HX:</B> This 78y/o RHM was referred for an episode of unusual behavior and RUE numbnes...",Radiology,"episodic mental status change numbness , chorea find exam . 78y rhm refer episode unusual behavior numbness . 9/91 , experience loss consciousness , generalize weakness , headache vomiting . evalu..."
815,"\n \n \n \n \n <B>PROCEDURE IN DETAIL: </B> Following premedication with Vistaril 50 mg and Atropine 0.4 mg IM, the patient received Versed 5.0 mg intravenously after Cetacaine spray to the poster...",Gastroenterology,"procedure follow premedication vistaril 50 mg atropine 0.4 mg , patient receive versed 5.0 mg cetacaine spray posterior palate . olympus video gastroscope introduce upper esophagus pass direct vis..."
816,\n \n \n \n \n <B>REASON FOR VISIT: </B> Overactive bladder with microscopic hematuria.\n \n <B>HISTORY OF PRESENT ILLNESS: </B> The patient is a 56-year-old noted to have microscopic hematuria wi...,Urology,reason overactive bladder microscopic hematuria . history present patient 56-year - old note microscopic hematuria overactive bladder . cystoscopy perform unremarkable . continue episode frequency...


<a id='parsing'></a>

# 2. Dependency Parsing

In dependency parsing, we try to use dependency-based grammars to analyze and infer both structure and semantic dependencies and relationships between tokens in a sentence. The basic principle behind a dependency grammar is that in any sentence in the language, all words except one, have some relationship or dependency on other words in the sentence. The word that has no dependency is called the root of the sentence. The verb is taken as the root of the sentence in most cases. All the other words are directly or indirectly linked to the root verb using links , which are the dependencies.

In [11]:
from spacy import displacy
text_tokenized = nlp(data.clean_text.iloc[1])

options = {"compact": True}
displacy.serve(text_tokenized, style="dep", options=options)


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


<a id='ner'></a>

# 3. Named Entity Recognition (NER)

In any text document, there are particular terms that represent specific entities that are more informative and have a unique context. These entities are known as named entities , which more specifically refer to terms that represent real-world objects like people, places, organizations, and so on, which are often denoted by proper names. A naive approach could be to find these by looking at the noun phrases in text documents. Named entity recognition (NER) , also known as entity chunking/extraction , is a popular technique used in information extraction to identify and segment the named entities and classify or categorize them under various predefined classes.

## we use the serve so that the below code can be opended in the same kernal and without going on to another page 

The quickest way to visualize Doc is to use displacy. serve . This will spin up a simple web server and let you view the result straight from your browser. displaCy can either take a single Doc or a list of Doc objects as its first argument.

## Always go for domain realted models as it would be great in the application of the NER

In [17]:
text_tokenized_orig = nlp(data.text.iloc[1])
#for ent in text_tokenized_orig_eng.ents:
#    print ("{} ---> {}".format(ent.text, ent.label_))
    
displacy.serve(text_tokenized_orig, style="ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


## Here we use the regular English Model and its not that great at sorting the drug names and diseases 

In [14]:
# English Language Model
nlp_eng = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)

In [15]:
text_tokenized_orig_eng = nlp_eng(data.text.iloc[1])
#for ent in text_tokenized_orig_eng.ents:
#    print ("{} ---> {}".format(ent.text, ent.label_))
    
displacy.serve(text_tokenized_orig_eng, style="ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


## References for further reading

<strong> POS tagging </strong>

* https://www.nltk.org/book/ch05.html

<strong> Medical named entity recognition </strong>

* https://github.com/kormilitzin/med7

* https://github.com/NLPatVCU/medaCy

* https://github.com/text-machine-lab/CliNER
